In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese_lccc/main')

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.2
GPU Enabled: False


In [4]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx


def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/lstm_pointer_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/lstm_pointer_export/1599195155/variables/variables


In [ ]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  for j in range(3):
    indices = [params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]
    print('Rank_{}:'.format(j+1), ' '.join(indices))
  print('%.2f sec' % (time.time() - t0))
  print()

Input:你好
Rank_1: 你 好
Rank_2: 你 好 ！
Rank_3: 你 也 好
1.08 sec

Input:早上好
Rank_1: 早 上 好
Rank_2: 早 上 好 ！
Rank_3: 早 安
0.05 sec

Input:晚上好
Rank_1: 晚 上 好
Rank_2: 晚 上 好 ！
Rank_3: 早 上 好
0.05 sec

Input:再见
Rank_1: 再 见
Rank_2: 再 见 ！
Rank_3: 哈 哈
0.19 sec

Input:好久不见
Rank_1: 好 久 不 见
Rank_2: 是 啊
Rank_3: 好 久 不 见 了
0.22 sec

Input:想死你了
Rank_1: 我 也 是
Rank_2: 来 吧 来 吧
Rank_3: 我 也 想 死
0.19 sec

Input:谢谢你
Rank_1: 不 客 气
Rank_2: 不 用 谢
Rank_3: 客 气
0.20 sec

Input:爱你
Rank_1: 爱 你
Rank_2: 我 也 是
Rank_3: 我 也 爱 你
0.20 sec

Input:你叫什么名字
Rank_1: 你 猜
Rank_2: 不 知 道
Rank_3: 我 不 知 道
0.24 sec

Input:你几岁了
Rank_1: 问 女 孩 子 年 龄 可 不 是 绅 士 所 为 哦 ~
Rank_2: 小 女 子 年 方 二 八 你 猜 我 多 大
Rank_3: 怎 么 能 问 一 个 淑 女 这 么 私 密 的 问 题 除 非 你 先 说
0.16 sec

Input:现在几点了
Rank_1: 1 1 点
Rank_2: 不 知 道
Rank_3: 一 点 多
0.22 sec

Input:今天天气怎么样
Rank_1: 下 雨 了
Rank_2: 今 天 下 雨 了
Rank_3: 今 天 天 气 很 好
0.16 sec

Input:我们现在在哪里
Rank_1: 在 这 里
Rank_2: 在 家
Rank_3: 在 哪 里
0.08 sec

Input:你能给我讲个笑话吗
Rank_1: 不 能
Rank_2: 可 以
Rank_3: 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈